# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import warnings
from sklearn.linear_model import LogisticRegression 
from sklearn import tree
from sklearn.metrics import accuracy_score

warnings.filterwarnings('ignore')

df=pd.read_csv('/datasets/users_behavior.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [2]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>Вывод: Получен датафрейм, без пропусков и прочих аномалий в данных. Все столбцы имеют нужный тип данных. Можно приступать к построению модели машинного обучения</font>
</div>

## Разбейте данные на выборки

In [4]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra'] 

In [5]:
features_train, features_test, target_train, target_test = train_test_split(features, target, 
                                                    train_size=0.6, 
                                                    random_state=12345)

In [6]:
features_test, features_valid, target_test, target_valid = train_test_split(features_test, target_test, 
                                                    train_size=0.5, 
                                                    random_state=12345,
                                                    )
print(f"Количество строк в y_test по классам: {np.bincount(target_test)}")
print(f"Количество строк в y_val по классам: {np.bincount(target_valid)}")

Количество строк в y_test по классам: [454 189]
Количество строк в y_val по классам: [440 203]


<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>В таком случае, делим сначала на 60(трейнинг)/40(тестинг), а затем валидационную и тестовую делим 50/50.</font>
</div>

## Исследуйте модели

<div class="alert alert-info">
 <font color='Blue'>
     Проверим логистическую регрессию с разными solver

In [7]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=100) 
model.fit(features_train, target_train) 
model.score(features_valid, target_valid) 

0.6842923794712286

In [8]:
model = LogisticRegression(random_state=12345, solver='sag', max_iter=1000) 
model.fit(features_train, target_train) 
model.score(features_valid, target_valid) 


0.6842923794712286

In [9]:
model = LogisticRegression(random_state=12345, solver='saga', max_iter=10000) 
model.fit(features_train, target_train) 
model.score(features_valid, target_valid) 


0.687402799377916

In [10]:
model = LogisticRegression(random_state=12345, solver='newton-cg', max_iter=10000) 
model.fit(features_train, target_train) 
model.score(features_valid, target_valid) 


0.7387247278382582

In [11]:
model = LogisticRegression(random_state=12345, solver='liblinear', max_iter=1000) 
model.fit(features_train, target_train) 
model.score(features_valid, target_valid) 


0.6889580093312597

<div class="alert alert-info">
 <font color='Blue'>
Так как lbfgs оказывается наиболее быстрым и точным методом решения, остановимся на нем и выведем classification_report

<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>Попробовал перенести импорты в самое начало. Добавил цикл для логистический регрессии, получил 100 итераций методом Ньютона и точность 0,73</font>
</div>

In [12]:
best_solver = None
best_iter = 0
best_lr_model = None
best_result=0

for solver in ['lbfgs', 'saga', 'newton-cg', 'liblinear']:
    for m_iter in [100, 1000, 10000]:
        model = LogisticRegression(random_state=12345, solver=solver, max_iter=m_iter) 
        model.fit(features_train, target_train)
        result = model.score (features_valid, target_valid)
        if result >best_result:
                best_iter = m_iter
                best_result = result
                best_solver = solver
print(best_iter, best_result, best_solver)

100 0.7387247278382582 newton-cg


<div class="alert alert-info">
 <font color='Blue'>
     Деревья решений

In [13]:
best_result=0
best_depth = 0
best_model = None

for depth in range(1, 6):
    model = tree.DecisionTreeClassifier(random_state=21,max_depth=depth)
    model.fit(features_train, target_train)
    result=model.score (features_valid, target_valid)
    if result >best_result:
        best_model = model
        best_result = result
        best_depth = depth
print(best_result, best_depth)

0.7822706065318819 5


<div class="alert alert-info">
 <font color='Blue'>
     В результате, дерево решений оказывается наилучшим выбором.

## Проверьте модель на тестовой выборке

Применим функцию score к лучшей модели(дерево решений с глубиной=3)

In [14]:
print(best_model.score(features_test,target_test))

0.7791601866251944


<div class="alert alert-info">
 <font color='Blue'>
Возможно, другое процентное соотношение выборок дало бы более оптимальные результаты

## (бонус) Проверьте модели на адекватность

<div class="alert alert-info">
 <font color='Blue'>
     Попробуем предсказывать всегда только 0, и сравним результаты с полученной моделью дерева решений.
     Для этого делаю срез - выбираю только те значения, исходной выборки, в которых is_ultra равняется 0.

In [15]:
df_only_zeros=df.query('is_ultra==0')
df_only_zeros.head()
model_always_zeros=tree.DecisionTreeClassifier(random_state=21,max_depth=3)
features_only_zeros = df_only_zeros.drop(['is_ultra'], axis=1)
target_only_zeros = df_only_zeros['is_ultra'] 

In [16]:
features_train_zeros, features_test_zeros, target_train_zeros, target_test_zeros = train_test_split(features_only_zeros, target_only_zeros, 
                                                    train_size=0.25, 
                                                    random_state=12345)

<div class="alert alert-info">
 <font color='Blue'>
     Обучаем модель на одних нулях(чтобы она всегда предсказывала ноль)

In [17]:
model_always_zeros.fit(features_train_zeros, target_train_zeros)

DecisionTreeClassifier(max_depth=3, random_state=21)

<div class="alert alert-info">
 <font color='Blue'>
     В качестве тестовой выборке берется та же самая, что и была для исходной модели

In [18]:
model_always_zeros.score(features_test,target_test)

0.7060653188180405

<div class="alert alert-info">
 <font color='Blue'>
     Вывод: Точность хуже на 10%

<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>В ходе выполнения этого задания, оказалось, что модель, ставящая единицы выдает точность 16%, когда модель, дающая лишь нули - 83%, что лучше. Но это происходит из-за того, что у нас попросту 83% нули и 16% единицы. Возможно, я неверно понял задание. Второй подход:
    Если считать именно метрику accuracy_score(target_test,dummy_prediction) из sklearn, то ее точность 70%. Это на 8% уступает полученной выше модели</font>
</div>

In [19]:
size=len(target_test)
print(size)
dummy_prediction=np.zeros(size)

643


In [20]:
dummy_prediction_ones=[]
for i in range(len(target_test)):
    dummy_prediction_ones.append(1)

In [21]:
best_model.score(features_test,dummy_prediction_ones)

0.16640746500777606

<div style="background: #B0E0E6; padding: 5px; border: 1px solid SteelBlue; border-radius: 5px;">
    <font color='4682B4'><u><b>КОММЕНТАРИЙ СТУДЕНТА</b></u></font>
    <br />
    <font color='4682B4'>accuracy_score:</font>
</div>

In [22]:
accuracy_score(target_test,dummy_prediction)

0.7060653188180405

## Вывод

В ходе проделанной работы была построена модель для совета пользователям тарифа(то есть решалась задача Классификации). Модель работает с точность 78% и прошла проверку на адекватность. 

Была протестирована логистическая регрессия и дерево решений, были перепробованы различные варианты гиперпараметров. Конкретно в этой задаче Дерево Решений с глубиной 5 оказалось лучшим решением.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
